In [1]:
print('jmd shree ganesha')

jmd shree ganesha


In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.vectorstores import chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import ctransformers
import os


In [3]:
dataset = r"/home/pheonix/Aster/FAQ-Chatbot/data"

In [4]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf(dataset)

In [6]:
def text_splitter(data_pdf):
    splitter = RecursiveCharacterTextSplitter(chunk_size=150, chunk_overlap=10)
    chunks = splitter.split_documents(data_pdf)
    
    return chunks

In [7]:
text_split_chunks = text_splitter(extracted_data)

In [8]:
text_split_chunks

[Document(page_content='Yes, you can book appointments and many treatments directly with us. For most procedures, a \nGP referral is required.', metadata={'source': '/home/pheonix/Aster/FAQ-Chatbot/data/dataset.pdf', 'page': 0}),
 Document(page_content='If you are insured, we also recommend that you check with your insurer that you are covered to \nsee your preferred consultant at our hospital.', metadata={'source': '/home/pheonix/Aster/FAQ-Chatbot/data/dataset.pdf', 'page': 0}),
 Document(page_content='You can easily book an appointment online with many of our consultants. To find a consultant', metadata={'source': '/home/pheonix/Aster/FAQ-Chatbot/data/dataset.pdf', 'page': 0}),
 Document(page_content='near you, please clic k here. For more information, please call 0330 173 8200.', metadata={'source': '/home/pheonix/Aster/FAQ-Chatbot/data/dataset.pdf', 'page': 0}),
 Document(page_content='We are proud to work with expert health professionals who specialize in a wide range of', metadat

In [9]:
print(len(text_split_chunks))

84


In [10]:
def text_to_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = text_to_embeddings()

/home/pheonix/Aster/FAQ-Chatbot/chatbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
persist_directory = 'chat'

vectordb = chroma.Chroma.from_documents(documents=text_split_chunks, embedding=embeddings, persist_directory=persist_directory)

In [14]:
vectordb.persist()
vectordb = None

In [15]:
vectordb = chroma.Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [16]:
retriever = vectordb.as_retriever()

In [17]:
docsearch = retriever.get_relevant_documents('Do I need a referral from my GP?')

/home/pheonix/Aster/FAQ-Chatbot/chatbot/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [18]:
len(docsearch)
print(docsearch)

[Document(page_content='A GP referral is required for most procedures as this will ensure that you are booked in for the', metadata={'page': 0, 'source': '/home/pheonix/Aster/FAQ-Chatbot/data/dataset.pdf'}), Document(page_content='A GP referral is required for most procedures as this will ensure that you are booked in for the', metadata={'page': 0, 'source': '/home/pheonix/Aster/FAQ-Chatbot/data/dataset.pdf'}), Document(page_content='A GP referral is required for most procedures as this will ensure that you are booked in for the', metadata={'page': 0, 'source': '/home/pheonix/Aster/FAQ-Chatbot/data/dataset.pdf'}), Document(page_content='appropriate speciality. There may be some treatme nts where a referral is not needed. For further', metadata={'page': 0, 'source': '/home/pheonix/Aster/FAQ-Chatbot/data/dataset.pdf'})]


In [19]:
retriever = vectordb.as_retriever(search_kwargs={'k':4})

In [20]:
retriever.search_type

'similarity'

In [21]:
retriever.search_kwargs

{'k': 4}

In [22]:
prompt_template="""
Use the following pieces of information to answer the user's question. You are a FAQ chatbot for Nuffield Health Hostipal.
If you don't know the answer, just say that to contact the helpline number. Not to ask for phone number

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [23]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [24]:
llm=ctransformers.CTransformers(model=r"model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.7})

RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-6638b27d-275415c875f3d34560c67313;8d553348-fcc6-4a31-b978-c627c6cc486c)

Repository Not Found for url: https://huggingface.co/api/models/model/llama-2-7b-chat.ggmlv3.q4_0.bin/revision/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 5}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])